<a href="https://colab.research.google.com/github/ZlatanSU87/Graduate-work.-Forecast-for-the-cafe-chain/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.9 MB/s eta 0:00:00


Импортируем библиотеки, читаем df

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

import re
import unicodedata


from rapidfuzz import fuzz
import networkx as nx
from collections import Counter

df = pd.read_csv("/content/drive/MyDrive/Cafe_prediction/cafe_with_weather.csv")

In [ ]:
df.shape

(2514160, 30)

В признаках "Блюдо", "Группа блюда" имеются идентичные названия, которые записаны с некоторым отличием. Выявим их и уберем подобные дубликаты.

In [ ]:
def normalize_text_safe(s):
    if s is None:
        return ""
    s = str(s)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower()
    # оставить только буквенно-цифровые, пробелы, дефисы и подчёркивание,
    # остальные символы заменить на пробел
    s = ''.join(ch if (ch.isalnum() or ch in ' _-') else ' ' for ch in s)
    # привести дефисы к пробелам и схлопнуть пробелы
    s = re.sub(r"[-–—]+", " ", s)
    s = re.sub(r"s+", " ", s).strip()
    return s


In [ ]:
def _cluster_and_build_mapping(values_series: pd.Series,
                               normalize_func,
                               threshold: int = 90,
                               scorer=fuzz.token_set_ratio):
    """
    Для одной серии: строит граф похожих нормализованных значений и возвращает mapping
    нормализованное значение -> каноническое нормализованное значение.
    """
    # Оставляем строки, приводим к str и нормализуем
    orig = values_series.astype(str).fillna("").tolist()
    norm_all = [normalize_func(x) for x in orig]

    # считаем частоты нормализованных значений в исходных данных (для выбора каноничного)
    freq = Counter(norm_all)

    # уникальные нормализованные значения в порядке появления
    unique_norm = []
    seen = set()
    for v in norm_all:
        if v not in seen and v != "":
            unique_norm.append(v)
            seen.add(v)

    # защита от пустого набора
    if not unique_norm:
        return {}, []

    # build similarity graph
    G = nx.Graph()
    G.add_nodes_from(unique_norm)

    n = len(unique_norm)
    # предупреждение о сложности: n^2 сравнения
    if n > 3000:
        # можно изменить порог или использовать блокировку; здесь просто предупреждаем
        print(f"Warning: {n} unique values -> O(n^2) сравнения (медленно).")

    for i, a in enumerate(unique_norm):
        for b in unique_norm[i+1:]:
            try:
                score = scorer(a, b)
            except Exception:
                score = 0
            if score >= threshold:
                G.add_edge(a, b)

    clusters = list(nx.connected_components(G))

    # выбрать канонические имена по частоте (если равны, по длине)
    mapping = {}
    for cluster in clusters:
        # cluster — множество нормализованных строк
        canonical = max(cluster, key=lambda x: (freq.get(x, 0), -len(x)))
        for name in cluster:
            mapping[name] = canonical

    return mapping, clusters

def canonicalize_columns(df: pd.DataFrame,
                         columns,
                         normalize_func=normalize_text_safe,
                         threshold: int = 90,
                         scorer=fuzz.token_set_ratio,
                         clean_suffix="_clean",
                         canonical_suffix="_canonical"):
    """
    Обрабатывает список колонок: создаёт для каждой <col>_clean и <col>_canonical.
    Возвращает (df_modified, mappings), где mappings — dict: column -> mapping dict.
    """
    df = df.copy()
    mappings = {}

    for col in columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found in dataframe")

        clean_col = f"{col}{clean_suffix}"
        canon_col = f"{col}{canonical_suffix}"

        # Создаём нормализованный столбец
        df[clean_col] = df[col].astype(str).map(normalize_func)

        mapping, clusters = _cluster_and_build_mapping(df[col], normalize_func, threshold, scorer)

        # Применяем mapping на основе нормализованного столбца
        df[canon_col] = df[clean_col].map(mapping).fillna(df[clean_col])

        mappings[col] = {
                        "mapping": mapping,
            "clusters": clusters
        }

    return df, mappings

In [ ]:
# Обработать одновременно "Блюдо" и "Группа блюда"
df, maps = canonicalize_columns(df, ["Блюдо", "Группа блюда"], threshold=90)

In [ ]:
print (df["Блюдо_canonical"].nunique(), df["Группа блюда_canonical"].nunique())
print (df["Блюдо"].nunique(), df["Группа блюда"].nunique())

696 62
979 74


In [ ]:
# В ручную посмотрим список 'Блюдо_canonical' и удалим явно не нужные названия
wtf = ['прибор 1шт', 'пакет с ручками ','греть чуть_чуть', 'снят с продажи', 'cтакан молока ребенку', '']
display(df.query('Блюдо_canonical in @wtf').shape[0])
# 9497 - не критично, удаляем
df = df.query('not (Блюдо_canonical in @wtf)')

9497

In [ ]:
# Заменим nan на правильный формат
cols = ['Блюдо_canonical', 'Группа блюда_canonical']
df[cols] = df[cols].replace(r'^s*nans*$', np.nan, regex=True)

Проведем предобработку

In [ ]:
# процент пропусков по колонкам
miss = df.isna().mean().sort_values(ascending=False)
display(miss[miss > 0].head(50))  # показать только колонки с пропусками


,0
Группа блюда_canonical,0.047209
Группа блюда,0.047209
Блюдо_canonical,0.014128
Блюдо,0.014128
Количество блюд,0.004208
Тип оплаты,0.000198


In [ ]:
# Удалим дублирующие и явно не нужные признаки после предварительного преобразования
df = df.drop(['Блюдо', 'Группа блюда', 'Блюдо_clean', 'Группа блюда_clean', 'Дата', 'date', 'time'], axis=1)

In [ ]:
# Нормализация названий блюд (на всякий случай)
def normalize_name(s):
    if pd.isna(s):
        return np.nan
    s = str(s)
    s = unicodedata.normalize('NFKC', s)
    s = s.lower().strip()
    s = re.sub(r's+', ' ', s)            # мн. пробелы -> один
    s = re.sub(r'[^ws]', '', s)         # удалить пунктуацию (опционально)
    return s

df['dish_norm'] = df['Блюдо_canonical'].apply(normalize_name)


In [ ]:
# Построим маппинг по наиболее частой (mode) группе для каждого блюда, в подавляющем большинстве случаев блюдо
# связано с одной группой.
mapping = (
    df.dropna(subset=['dish_norm','Группа блюда_canonical'])
      .groupby('dish_norm')['Группа блюда_canonical']
      .agg(lambda x: x.value_counts().index[0])
      .to_dict()
)

In [ ]:
#  fuzzy matching (rapidfuzz)
# если в датасете много опечаток/вариаций, сравним незнакомые dish_norm с базой известных dish_norm и возьмем
# ближайшее совпадение с порогом.
from rapidfuzz import process, fuzz

known = list(mapping.keys())  # список известных dish_norm
def fuzzy_map(name, scorer=fuzz.token_sort_ratio, score_cutoff=85):
    if pd.isna(name):
        return None
    res = process.extractOne(name, known, scorer=scorer, score_cutoff=score_cutoff)
    if res:
        match_name, score, _ = res
        return mapping.get(match_name)
    return None

mask2 = df['Группа блюда_canonical'].isna() & df['dish_norm'].notna()
df.loc[mask2, 'Группа блюда_canonical'] = df.loc[mask2, 'dish_norm'].apply(fuzzy_map)


In [ ]:
df['Группа блюда_canonical'].isna().mean()



np.float64(0.002427472278705758)

In [ ]:
# С учётом относительно небольшого количества пропусков по ряду признаков, заполним их медианой и модой.
df['Количество блюд'] = df['Количество блюд'].fillna(df['Количество блюд'].median()).round().astype(int)
df['Тип оплаты'] = df['Тип оплаты'].fillna(df['Тип оплаты'].mode().iloc[0])

In [ ]:
miss = df.isna().mean().sort_values(ascending=False)
display(miss[miss > 0].head(50))  # показать только колонки с пропусками

,0
dish_norm,0.014128
Блюдо_canonical,0.014128
Группа блюда_canonical,0.002427


In [ ]:
df['Блюдо_canonical'] = df['Блюдо_canonical'].fillna(
    df.groupby('Торговое предприятие')['Блюдо_canonical']
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
).fillna('UNKNOWN')

df['Группа блюда_canonical'] = df['Группа блюда_canonical'].fillna(
    df.groupby('Торговое предприятие')['Группа блюда_canonical']
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
).fillna('UNKNOWN')


In [ ]:
# Удаляем не нужный признак
df = df.drop('dish_norm', axis=1)

In [ ]:
# Проверим пустые значения во всем df
# замениv пустые и пробельные строки на NaN по всему df
df_clean = df.replace(r'^s*$', np.nan, regex=True)

# теперь стандартные проверки
print(df_clean.isna().sum().sum())
print('Есть пропуски?', df_clean.isna().values.any())
print('Колонки с пропусками:', df_clean.columns[df_clean.isna().any()].tolist())


0
Есть пропуски? False
Колонки с пропусками: []


In [ ]:
df.head(2)

,Время открытия,Количество блюд,Тип оплаты,Торговое предприятие,"Сумма со скидкой, р.","Сумма без скидки, р.",Наценка(%),"Себестоимость, р.",Себестоимость(%),holiday_name,...,wind_max,humidity,snow,rain,weathercode,sunrise_ts,sunset_ts,is_daytime,Блюдо_canonical,Группа блюда_canonical
0,2021-04-06 15:48:05,1,Наличные,Энгельса,60.0,60.0,1.1761,27.5725,0.4595,Рабочий день,...,49.0,76.583333,0.0,2.5,51,2021-04-06 05:47:00+03:00,2021-04-06 19:42:00+03:00,True,капучино,кофе
1,2021-04-06 15:48:05,1,Наличные,Энгельса,90.0,90.0,1.5714,35.0000,0.3889,Рабочий день,...,49.0,76.583333,0.0,2.5,51,2021-04-06 05:47:00+03:00,2021-04-06 19:42:00+03:00,True,трубочка сырная,десерты выпечка


Проведем типизацию и экономию памяти

• Преобразуем колонки с подстроковыми категориями в category.

• Приведем целые/вещественные колонки к меньшим типам (downcast).

• Проверим наличие отрицательных или нулевых значений там, где это нежелательно.

In [ ]:
def reduce_mem_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type == 'object':
            df[col] = df[col].astype('category')
        elif col_type.kind in 'iuf':  # Int/float
            if col_type.kind == 'i':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            else:
                df[col] = pd.to_numeric(df[col], downcast='float')
    return df

df = reduce_mem_usage(df)
# некоторые колонки пока оставим object

In [ ]:
# Преобразование времени и таймзоны
df['Время открытия'] = pd.to_datetime(df['Время открытия'])
df['Время открытия'] = df['Время открытия'].dt.tz_localize('Europe/Moscow')  # если это действительно локальное время

# если некоторые колонки (sunrise_ts/sunset_ts) уже с +03:00, pd.to_datetime корректно распарсит
df['sunrise_ts'] = pd.to_datetime(df['sunrise_ts'])
df['sunset_ts'] = pd.to_datetime(df['sunset_ts'])


In [ ]:
# Базовые проверки качества данных
# • Проверим NaN по колонкам; для временно-зависимых признаков (погодa) разумно агрегировать/заполнить на уровне дня.
# • Проверим отрицательные/нулевые значения в Количество блюд, Сумма со скидкой, р..
# • Проверим аномальные большие значения (напр. > 99 перцентиля) — возможно возвраты/ошибки.

# пропуски и базовая статистика
print(df[['Количество блюд','Сумма со скидкой, р.']].describe())
print(df[['Количество блюд','Сумма со скидкой, р.']].isna().sum())

# отрицательные/нулевые
print((df['Количество блюд'] <= 0).sum(), (df['Сумма со скидкой, р.'] <= 0).sum())

# проверка аномалий
q99 = df['Сумма со скидкой, р.'].quantile(0.99)
print('99-й перцентиль суммы:', q99)

       Количество блюд  Сумма со скидкой, р.
count     2.504663e+06          2.504663e+06
mean      1.108451e+00          1.217435e+02
std       6.250215e-01          1.189850e+02
min      -2.200000e+01         -7.000000e+03
25%       1.000000e+00          7.000000e+01
50%       1.000000e+00          1.250000e+02
75%       1.000000e+00          1.700000e+02
max       9.000000e+01          4.488600e+04
Количество блюд         0
Сумма со скидкой, р.    0
dtype: int64
23092 462772
99-й перцентиль суммы: 398.0


In [ ]:

cols = ['Сумма со скидкой, р.', 'Количество блюд']

def sign_counts(s):
    return pd.Series({
        '<0':  (s < 0).sum(),
        '==0': (s == 0).sum(),
        '>0':  (s > 0).sum()
    })

result = df[cols].apply(sign_counts)
result

,"Сумма со скидкой, р.",Количество блюд
<0,12646,14828
==0,450126,8264
>0,2041891,2481571


In [ ]:
# Проконсультировавшись с админстратором кафе, с учетом того, что отрицательная "Сумма со скидкой, р."
# и "Количество блюд" - это возврат, целесообразно указанные данные удалить.

df = df[(df['Сумма со скидкой, р.'] >= 0) & (df['Количество блюд'] >= 0)]


In [ ]:
q99 = df['Сумма со скидкой, р.'].quantile(0.99)
df_high = df[df['Сумма со скидкой, р.'] > q99].sort_values('Сумма со скидкой, р.', ascending=False)
df_high.head(20)
# Данные 99 квантиля - групповые заказы, оставляем.

,Время открытия,Количество блюд,Тип оплаты,Торговое предприятие,"Сумма со скидкой, р.","Сумма без скидки, р.",Наценка(%),"Себестоимость, р.",Себестоимость(%),holiday_name,...,wind_max,humidity,snow,rain,weathercode,sunrise_ts,sunset_ts,is_daytime,Блюдо_canonical,Группа блюда_canonical
2304484,2025-07-06 19:07:46+03:00,1,Оплата по QR-коду,Свердлова,44886.00,44886.00,0.0000,0.000000,0.0000,Рабочий день,...,25.200001,70.916664,0.0,0.0,3,2025-07-06 03:11:00+03:00,2025-07-06 22:23:00+03:00,True,организация мероприятия,мероприятия
2182591,2025-05-24 19:54:57+03:00,1,Оплата по QR-коду,Свердлова,38590.00,38590.00,0.0000,0.000000,0.0000,Рабочий день,...,33.500000,74.958336,0.0,3.2,3,2025-05-24 03:27:00+03:00,2025-05-24 21:51:00+03:00,True,организация мероприятия,мероприятия
2400751,2025-08-11 13:31:08+03:00,1,Оплата по QR-коду,Свердлова,28725.42,28725.42,0.0000,0.000000,0.0000,Рабочий день,...,27.000000,85.541664,0.0,2.3,3,2025-08-11 04:38:00+03:00,2025-08-11 20:57:00+03:00,True,организация мероприятия,мероприятия
2302050,2025-07-05 17:01:13+03:00,1,Оплата по QR-коду,Свердлова,25925.00,25925.00,0.0000,0.000000,0.0000,Рабочий день,...,62.299999,65.958336,0.0,0.3,3,2025-07-05 03:09:00+03:00,2025-07-05 22:25:00+03:00,True,организация мероприятия,мероприятия
62419,2021-07-17 23:57:19+03:00,72,Оплата по QR-коду,Энгельса,17244.13,17244.13,9.1927,1695.327759,0.0981,Рабочий день,...,29.500000,50.250000,0.0,0.0,0,2021-07-17 03:33:00+03:00,2021-07-17 22:03:00+03:00,False,капучино,алкоголь
325729,2022-06-27 22:57:38+03:00,90,Оплата по QR-коду,Энгельса,13500.00,13500.00,0.7064,7911.186035,0.5860,Рабочий день,...,33.500000,64.500000,0.0,0.0,0,2022-06-27 02:59:00+03:00,2022-06-27 22:32:00+03:00,False,пломбир кусочек,десерты выпечка
770345,2023-08-29 10:57:47+03:00,90,Оплата по QR-коду,Энгельса,13500.00,13500.00,7.0252,1682.201660,0.1246,Рабочий день,...,52.200001,71.916664,0.0,0.7,3,2023-08-29 05:24:00+03:00,2023-08-29 20:02:00+03:00,True,капучино,молоко 240 гр
770342,2023-08-29 10:57:47+03:00,89,Оплата по QR-коду,Энгельса,12905.00,12905.00,4.4980,2347.197021,0.1819,Рабочий день,...,52.200001,71.916664,0.0,0.7,3,2023-08-29 05:24:00+03:00,2023-08-29 20:02:00+03:00,True,молоко,молоко 180гр
332168,2022-07-03 23:44:48+03:00,78,Оплата по QR-коду,Энгельса,11775.00,11775.00,0.7538,6713.919434,0.5702,Рабочий день,...,30.200001,77.583336,0.0,1.1,3,2022-07-03 03:05:00+03:00,2022-07-03 22:27:00+03:00,False,капучино,десерты выпечка
331401,2022-07-02 23:56:07+03:00,74,Оплата по QR-коду,Энгельса,11100.00,11100.00,0.7064,6504.755371,0.5860,Рабочий день,...,29.500000,66.958336,0.0,0.4,0,2022-07-02 03:04:00+03:00,2022-07-02 22:28:00+03:00,False,пломбир кусочек,десерты выпечка


In [ ]:
# полные дубликаты строк
dups_full = df.duplicated().sum()
print("Полных дубликатов строк:", dups_full)
# если есть — удалить
if dups_full > 0:
    df = df.drop_duplicates().reset_index(drop=True)
    print("Удалили полные дубликаты. Новая форма:", df.shape)

# частые дублировки на уровне позиции: одна и та же строка заказа повторяется?
cols_for_line_dup = ['Время открытия','Блюдо_canonical','Группа блюда_canonical','Количество блюд','Торговое предприятие','Сумма со скидкой, р.']
cols_present = [c for c in cols_for_line_dup if c in df.columns]
dups_line = df.duplicated(subset=cols_present).sum()
print("Дубликатов на уровне позиции (по ключевым колонкам):", dups_line)
# если много — можно также удалить, но сначала можно посмотреть несколько примеров
if dups_line > 0:
    display(df[df.duplicated(subset=cols_present, keep=False)].head(5))

Полных дубликатов строк: 1092
Удалили полные дубликаты. Новая форма: (2488537, 27)
Дубликатов на уровне позиции (по ключевым колонкам): 11899


,Время открытия,Количество блюд,Тип оплаты,Торговое предприятие,"Сумма со скидкой, р.","Сумма без скидки, р.",Наценка(%),"Себестоимость, р.",Себестоимость(%),holiday_name,...,wind_max,humidity,snow,rain,weathercode,sunrise_ts,sunset_ts,is_daytime,Блюдо_canonical,Группа блюда_canonical
327,2021-04-08 13:55:13+03:00,1,Банковские карты,Энгельса,96.5,96.5,0.4611,67.754997,0.6844,Рабочий день,...,46.799999,87.416664,4.13,2.17,73,2021-04-08 05:41:00+03:00,2021-04-08 19:47:00+03:00,True,капучино,1салаты
329,2021-04-08 13:55:13+03:00,1,Банковские карты,Энгельса,96.5,96.5,0.4612,67.754372,0.6844,Рабочий день,...,46.799999,87.416664,4.13,2.17,73,2021-04-08 05:41:00+03:00,2021-04-08 19:47:00+03:00,True,капучино,1салаты
414,2021-04-09 08:33:04+03:00,1,Банковские карты,Энгельса,60.0,60.0,2.3768,17.768393,0.2961,Рабочий день,...,64.400002,71.250000,0.63,0.87,0,2021-04-09 05:38:00+03:00,2021-04-09 19:50:00+03:00,True,капучино,кофе
415,2021-04-09 08:33:04+03:00,1,Банковские карты,Энгельса,60.0,60.0,1.9025,20.671579,0.3445,Рабочий день,...,64.400002,71.250000,0.63,0.87,0,2021-04-09 05:38:00+03:00,2021-04-09 19:50:00+03:00,True,капучино,кофе
778,2021-04-09 15:25:54+03:00,1,Банковские карты,Энгельса,60.0,60.0,2.3768,17.768393,0.2961,Рабочий день,...,64.400002,71.250000,0.63,0.87,0,2021-04-09 05:38:00+03:00,2021-04-09 19:50:00+03:00,True,капучино,кофе


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2488537 entries, 0 to 2488536
Data columns (total 27 columns):
 #   Column                  Dtype                        
---  ------                  -----                        
 0   Время открытия          datetime64[ns, Europe/Moscow]
 1   Количество блюд         int8                         
 2   Тип оплаты              category                     
 3   Торговое предприятие    category                     
 4   Сумма со скидкой, р.    float64                      
 5   Сумма без скидки, р.    float64                      
 6   Наценка(%)              float64                      
 7   Себестоимость, р.       float32                      
 8   Себестоимость(%)        float32                      
 9   holiday_name            category                     
 10  is_holiday              int8                         
 11  t_mean                  float32                      
 12  t_max                   float32                      
 1

In [ ]:
# полные дубликаты строк
dups_full = df.duplicated().sum()
print("Полных дубликатов строк:", dups_full)
# если есть — удалить
if dups_full > 0:
    df = df.drop_duplicates().reset_index(drop=True)
    print("Удалили полные дубликаты. Новая форма:", df.shape)

# частые дублировки на уровне позиции: одна и та же строка заказа повторяется?
cols_for_line_dup = ['Время открытия','Блюдо_canonical','Количество блюд','Торговое предприятие','Сумма со скидкой, р.']
cols_present = [c for c in cols_for_line_dup if c in df.columns]
dups_line = df.duplicated(subset=cols_present).sum()
print("Дубликатов на уровне позиции (по ключевым колонкам):", dups_line)
# если много — можно также удалить, но сначала можно посмотреть несколько примеров
if dups_line > 0:
    display(df[df.duplicated(subset=cols_present, keep=False)].head(5))


Полных дубликатов строк: 0
Дубликатов на уровне позиции (по ключевым колонкам): 15818


,Время открытия,Количество блюд,Тип оплаты,Торговое предприятие,"Сумма со скидкой, р.","Сумма без скидки, р.",Наценка(%),"Себестоимость, р.",Себестоимость(%),holiday_name,...,wind_max,humidity,snow,rain,weathercode,sunrise_ts,sunset_ts,is_daytime,Блюдо_canonical,Группа блюда_canonical
179,2021-04-07 15:33:45+03:00,1,Банковские карты,Энгельса,20.0,20.0,0.0000,0.000000,0.0000,Рабочий день,...,49.000000,72.375000,0.49,0.41,0,2021-04-07 05:44:00+03:00,2021-04-07 19:44:00+03:00,True,сироп,американо
180,2021-04-07 15:33:45+03:00,1,Банковские карты,Энгельса,20.0,20.0,1.1645,9.240000,0.4620,Рабочий день,...,49.000000,72.375000,0.49,0.41,0,2021-04-07 05:44:00+03:00,2021-04-07 19:44:00+03:00,True,сироп,допы
327,2021-04-08 13:55:13+03:00,1,Банковские карты,Энгельса,96.5,96.5,0.4611,67.754997,0.6844,Рабочий день,...,46.799999,87.416664,4.13,2.17,73,2021-04-08 05:41:00+03:00,2021-04-08 19:47:00+03:00,True,капучино,1салаты
329,2021-04-08 13:55:13+03:00,1,Банковские карты,Энгельса,96.5,96.5,0.4612,67.754372,0.6844,Рабочий день,...,46.799999,87.416664,4.13,2.17,73,2021-04-08 05:41:00+03:00,2021-04-08 19:47:00+03:00,True,капучино,1салаты
414,2021-04-09 08:33:04+03:00,1,Банковские карты,Энгельса,60.0,60.0,2.3768,17.768393,0.2961,Рабочий день,...,64.400002,71.250000,0.63,0.87,0,2021-04-09 05:38:00+03:00,2021-04-09 19:50:00+03:00,True,капучино,кофе


Сохраняем датасет

In [ ]:
df.to_csv('/content/drive/MyDrive/Cafe_prediction/cafe_preprocessing.csv', index=False)
print('Saved merged file to', '/content/drive/MyDrive/Cafe_prediction/cafe_preprocessing.csv')

Saved merged file to /content/drive/MyDrive/Cafe_prediction/cafe_preprocessing.csv
